# Spam Classifier: Preparing data

In [1]:
import os 

In [2]:
try:
    ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except Exception:
    ROOT_DIR = os.path.abspath('..')

In [3]:
data_path = os.path.join(ROOT_DIR, 'data')

data_folder_names = [os.path.join(data_path, file_name) for file_name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, file_name))]
data_folder_names

['d:\\repos\\spam-classifier\\data\\easy_ham',
 'd:\\repos\\spam-classifier\\data\\easy_ham_2',
 'd:\\repos\\spam-classifier\\data\\hard_ham',
 'd:\\repos\\spam-classifier\\data\\spam',
 'd:\\repos\\spam-classifier\\data\\spam_2']

In [4]:
import email
import email.policy

In [5]:
def get_files(folder_path: str) -> iter:
    return (file for file in os.listdir(folder_path))

In [6]:
def retrieve_email(file_path: str) -> 'bytes':
    with open(file_path, 'rb') as file:
        return email.parser.BytesParser(policy= email.policy.default).parse(file)

In [10]:
from concurrent import futures

In [19]:
def load_one_data(data_folder_path: str) -> list:
    return [retrieve_email(os.path.join(data_folder_path, file)) for file in get_files(data_folder_path)]

In [7]:
def load_data(data_folder_path: list) -> dict:
    emails = {}
    for folder_path in data_folder_path:
        folder_name = os.path.basename(folder_path)
        emails[folder_name] = [retrieve_email(os.path.join(folder_path, file)) for file in get_files(folder_path)]
    
    return emails

In [15]:
def load_data_parallel(data_folder_path: list) ->dict:
    emails = {}
    with futures.ThreadPoolExecutor(len(data_folder_path)) as executor:
        res = executor.map(load_one_data, data_folder_path)
    
    for folder_path, mails in zip(data_folder_path, res):
        folder_name = os.path.basename(folder_path)
        emails[folder_name] = mails

    return emails


In [20]:
emails = load_data_parallel(data_folder_names)
emails

{'easy_ham': [<email.message.EmailMessage at 0x24165426fa0>,
  ...],
 'easy_ham_2': [<email.message.EmailMessage at 0x24165413dc0>,
  ...],
 'hard_ham': [<email.message.EmailMessage at 0x24165409a90>,
 'spam': [<email.message.EmailMessage at 0x24165409880>,
 'spam_2': [<email.message.EmailMessage at 0x241651fbd90>,
  ...]}

In [8]:
# sequential
# emails = load_data(data_folder_names)
# emails

{'easy_ham': [<email.message.EmailMessage at 0x24151c0e910>,
  ...],
 'easy_ham_2': [<email.message.EmailMessage at 0x24162a50e80>,
  ...],
 'hard_ham': [<email.message.EmailMessage at 0x241639cf790>,
 'spam': [<email.message.EmailMessage at 0x24163b2e850>,
 'spam_2': [<email.message.EmailMessage at 0x241644855e0>,
  ...]}

In [21]:
print(emails['easy_ham'][1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/
